### import modul dan library

In [1]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os
import tensorflow_model_analysis as tfma
from tfx.proto import trainer_pb2
from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.components import Evaluator
from tfx.components import Pusher 
from tfx.proto import pusher_pb2 

### Mengatur dan mempersiapkan variabel untuk pipeline

In [2]:
BASE = "output"
PIPELINE_NAME = "depression-pipeline"
SCHEMA_PIPELINE_NAME = "depression-tfdv-schema"

PIPELINE_ROOT = os.path.join(BASE, PIPELINE_NAME)

METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

SERVING_MODEL_DIR = os.path.join(BASE, 'serving_model')

In [3]:
DATA_ROOT = "data"

In [4]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

### Tahapan Data Ingestion

Mengimpor dan membagi data untuk pipeline

In [5]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [6]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

Menghasilkan dan menampilkan statistik deskriptif dari data

In [7]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
 
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [8]:
interactive_context.show(statistics_gen.outputs["statistics"])

Membuat skema berdasarkan statistik

In [9]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs["statistics"]
)
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [10]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'Academic Pressure',FLOAT,required,,-
'Age',FLOAT,required,,-
'CGPA',FLOAT,required,,-
'City',STRING,required,,'City'
'Degree',STRING,required,,'Degree'
'Depression',INT,required,,-
'Dietary Habits',STRING,required,,'Dietary Habits'
'Family History of Mental Illness',STRING,required,,'Family History of Mental Illness'
'Financial Stress',FLOAT,required,,-


,Values
Domain,
'City',"'3.0', 'Agra', 'Ahmedabad', 'Bangalore', 'Bhopal', 'Chennai', 'City', 'Delhi', 'Faridabad', 'Gaurav', 'Ghaziabad', 'Harsh', 'Harsha', 'Hyderabad', 'Indore', 'Jaipur', 'Kalyan', 'Kanpur', 'Kolkata', 'Less Delhi', 'Lucknow', 'Ludhiana', 'M.Com', 'Meerut', 'Mihir', 'Mira', 'Mumbai', 'Nagpur', 'Nalini', 'Nalyan', 'Nashik', 'Patna', 'Pune', 'Rajkot', 'Rashi', 'Reyansh', 'Saanvi', 'Srinagar', 'Surat', 'Thane', 'Vaanya', 'Vadodara', 'Varanasi', 'Vasai-Virar', 'Visakhapatnam', 'Bhavna', 'Khaziabad', 'Kibara', 'Less than 5 Kalyan', 'M.Tech', 'ME', 'Nandini'"
'Degree',"'B.Arch', 'B.Com', 'B.Ed', 'B.Pharm', 'B.Tech', 'BA', 'BBA', 'BCA', 'BE', 'BHM', 'BSc', 'Class 12', 'LLB', 'LLM', 'M.Com', 'M.Ed', 'M.Pharm', 'M.Tech', 'MA', 'MBA', 'MBBS', 'MCA', 'MD', 'ME', 'MHM', 'MSc', 'Others', 'PhD'"
'Dietary Habits',"'Healthy', 'Moderate', 'Others', 'Unhealthy'"
'Family History of Mental Illness',"'No', 'Yes'"
'Gender',"'Female', 'Male'"
'Have you ever had suicidal thoughts ?',"'No', 'Yes'"
'Profession',"'Architect', 'Chef', 'Civil Engineer', 'Content Writer', 'Digital Marketer', 'Doctor', 'Educational Consultant', 'Entrepreneur', 'Lawyer', 'Pharmacist', 'Student', 'Teacher', 'Manager', 'UX/UI Designer'"
'Sleep Duration',"'5-6 hours', '7-8 hours', 'Less than 5 hours', 'More than 8 hours', 'Others'"


Mengecek anomali di dalam data

In [11]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [12]:
interactive_context.show(example_validator.outputs['anomalies'])

### Tahapan Data Preprocessing

Membuat dan menyimpan fungsi transform untuk preprocessing data, dengan tahapan:
1. Membagi setiap fitur antara kategorikal, numerik, dan label.
2. Tansformasi nama fitur.
3. Fitur Kategorikal akan diubah menjadi integer, lalu di convert ke one hot encoding.
4. Fitur Numerik akan di normalisasikan ke skalama 0-1.
5. Fitur Label akan dipertahankan sebagai int64

In [13]:
TRANSFORM_MODULE_FILE = "depression_transform.py"

In [14]:
%%writefile {TRANSFORM_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft
 
CATEGORICAL_FEATURES = {
    "Dietary Habits": 4,
    "Family History of Mental Illness": 2,
    "Have you ever had suicidal thoughts ?": 2,
    "Sleep Duration": 5
}
NUMERICAL_FEATURES = [
    "Academic Pressure",
    "CGPA",
    "Financial Stress",
    "Study Satisfaction",
    "Work/Study Hours"
]
LABEL_KEY = "Depression"
 
def transformed_name(key):
    return key + "_xf"
 
def convert_num_to_one_hot(label_tensor, num_labels=2):
    one_hot_tensor = tf.one_hot(label_tensor, num_labels)
    return tf.reshape(one_hot_tensor, [-1, num_labels])
 
def preprocessing_fn(inputs):
    outputs = {}
    
    for key in CATEGORICAL_FEATURES:
        dim = CATEGORICAL_FEATURES[key]
        int_value = tft.compute_and_apply_vocabulary(
            inputs[key], top_k=dim + 1
        )
        outputs[transformed_name(key)] = convert_num_to_one_hot(
            int_value, num_labels=dim + 1
        )
    
    for feature in NUMERICAL_FEATURES:
        outputs[transformed_name(feature)] = tft.scale_to_0_1(inputs[feature])
    
    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)
    
    return outputs

Overwriting depression_transform.py


Menjalankan proses transform untuk preprocessing data.

In [15]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: output\depression-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\de0abb38a7014420b7bd3d12e479cee8\assets


INFO:tensorflow:Assets written to: output\depression-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\de0abb38a7014420b7bd3d12e479cee8\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

### Tahap Pengembangan Model

Membuat modul file tuner yang akan melakukan proses hyperparameters untuk model, dimana terdapat proses:
1. Kompresi data input.
2. Menyipkan dan preprocessing data input.
3. Fungsi model builder yang berisi struktur model dengan pilihan hyperparameters.
4. Fungsi run tuner yang akan melakukan proses menentukan hyperparameters terbaik berdasarkan data dan struktur model.

In [16]:
TUNER_MODULE_FILE = "depression_tuner.py"

In [17]:
%%writefile {TUNER_MODULE_FILE}
from typing import NamedTuple, Dict, Text, Any
from tfx.components.trainer.fn_args_utils import FnArgs
from kerastuner.engine import base_tuner
from kerastuner import Hyperband, Objective
import tensorflow as tf
import tensorflow_transform as tft
import json
import os

from depression_transform import (
    CATEGORICAL_FEATURES,
    LABEL_KEY,
    NUMERICAL_FEATURES,
    transformed_name
)

TunerResult = NamedTuple("TunerResult", [
    ("tuner", base_tuner.BaseTuner),
    ("fit_kwargs", Dict[Text, Any]),
])

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_binary_accuracy",
    mode="max",
    verbose=1,
    patience=5,
)

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type="GZIP")

def input_fn(file_pattern, tf_transform_output, batch_size=64):
    """Generates features and labels for tuning/training."""
    transformed_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )
    
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transformed_feature_spec,
        reader=gzip_reader_fn,
        label_key=transformed_name(LABEL_KEY),
    )
    
    return dataset

def model_builder(hp):
    input_features = []

    for key, dim in CATEGORICAL_FEATURES.items():
        input_features.append(
            tf.keras.Input(shape=(dim + 1,), name=transformed_name(key))
        )
    
    for feature in NUMERICAL_FEATURES:
        input_features.append(
            tf.keras.Input(shape=(1,), name=transformed_name(feature))
        )
    
    concatenate = tf.keras.layers.concatenate(input_features)
    
    num_layers = hp.Int('num_layers', min_value=1, max_value=3, step=1)
    deep = concatenate
    
    for i in range(num_layers):
        units = hp.Int(f'units_{i}', min_value=16, max_value=256, step=16)
        deep = tf.keras.layers.Dense(units, activation='relu')(deep)
        dropout_rate = hp.Float(f'dropout_{i}', min_value=0.0, max_value=0.5, step=0.1)
        if dropout_rate > 0:
            deep = tf.keras.layers.Dropout(dropout_rate)(deep)
    
    outputs = tf.keras.layers.Dense(1, activation="sigmoid")(deep)
    model = tf.keras.models.Model(inputs=input_features, outputs=outputs)
    
    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss="binary_crossentropy",
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )
    
    return model

def tuner_fn(fn_args: FnArgs):
    transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    train_data = input_fn(fn_args.train_files[0], transform_output)
    eval_data = input_fn(fn_args.eval_files[0], transform_output)

    tuner = Hyperband(
        hypermodel=lambda hp: model_builder(hp),
        objective=Objective('val_binary_accuracy', direction='max'),
        max_epochs=5,
        factor=3,
        directory=fn_args.working_dir,
        project_name="depression_tuner",
    )

    tuner.search(
        x=train_data,
        validation_data=eval_data,
        steps_per_epoch=50,
        validation_steps=50,
        callbacks=[early_stopping],
        epochs=5
    )
    
    best_hps = tuner.get_best_hyperparameters(1)[0]
    best_hps_values = best_hps.values
    
    with open(os.path.join('output', "best_hyperparameters.json"), "w", encoding="utf-8") as f:
        json.dump(best_hps_values, f)
    
    print(f"Best hyperparameters: {best_hps_values}")

    return TunerResult(
        tuner=tuner,
        fit_kwargs={
            "callbacks": [early_stopping],
            "x": train_data,
            "validation_data": eval_data,
            "steps_per_epoch": 50,
            "validation_steps": 50,
        },
    )

Overwriting depression_tuner.py


Menjalankan proses tuner untuk menentukan hyperparameters terbaik

In [18]:
tuner = Tuner(
    module_file=os.path.abspath(TUNER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=100),
    eval_args=trainer_pb2.EvalArgs(num_steps=50),
    custom_config={
        'tuner_epochs': 50,
    }
)

interactive_context.run(tuner)

Trial 13 Complete [00h 00m 05s]
val_binary_accuracy: 0.8090624809265137

Best val_binary_accuracy So Far: 0.8424999713897705
Total elapsed time: 00h 00m 56s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Results summary
Results in output\depression-pipeline\.temp\6\depression_tuner
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 2
units_0: 192
dropout_0: 0.2
learning_rate: 0.004311196053909165
tuner/epochs: 5
tuner/initial_epoch: 2
tuner/bracket: 1
tuner/round: 1
units_1: 16
dropout_1: 0.0
tuner/trial_id: 0000
Score: 0.8424999713897705
Trial summary
Hyperparameters:
num_layers: 3
units_0: 192
dropout_0: 0.2
learning_rate: 0.0010267634344924114
units_1: 144
dropout_1: 0.2
units_2: 144
dropout_2: 0.1
tuner/epochs: 5
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.8418750166893005
Trial summary
Hyperparameters:
num_layers: 2
units_0: 64
dropout_0: 0.0
learning_rate: 0.0038803956106591635
units_1: 256
dropout_1: 0.1
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.8396875262260437
Trial summary
Hyperparameters:
num_layers: 1
units_0: 192
dropout_0: 0.2
learning_rate: 0.008975496460951405
units_1: 96
dropout_1: 0.5
units_2: 19

ExecutionResult(
    component_id: Tuner
    execution_id: 6
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={})
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}))

Membuat modul trainer yang melakukan proses training model, yang mencakup:
1. Fungsi untuk mempersiapkan model agar bisa menerima input dalam bentuk tf.Example.
2. Fungsi Build Model yang membangun model berdasarkan hyperparameters terbaik yang didapat dari hasil tuner.
3. Fungsi Run yang terdiri dari pengambilan dataset, membangun model berdasarkan hp terbaik, penentuan model fit, dan menyimpan model.

In [19]:
TRAINER_MODULE_FILE = "depression_trainer.py"

In [20]:
%%writefile {TRAINER_MODULE_FILE}
import os 
from typing import Dict, Text, Any
from tfx.components.trainer.fn_args_utils import FnArgs
import tensorflow as tf
import tensorflow_transform as tft
import kerastuner
import json

from depression_transform import (
    CATEGORICAL_FEATURES,
    LABEL_KEY,
    NUMERICAL_FEATURES,
    transformed_name
)
from depression_tuner import (gzip_reader_fn, input_fn)

def get_serve_tf_examples_fn(model, tf_transform_output):
    """Returns a function that parses a serialized tf.Example."""
 
    model.tft_layer = tf_transform_output.transform_features_layer()
 
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        """Returns the output to be used in the serving signature."""
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)
        parsed_features = tf.io.parse_example(
            serialized_tf_examples, feature_spec
        )
 
        transformed_features = model.tft_layer(parsed_features)
 
        outputs = model(transformed_features)
        return {"outputs": outputs}
 
    return serve_tf_examples_fn

def build_model(hparams):
    input_features = []

    for key, dim in CATEGORICAL_FEATURES.items():
        input_features.append(
            tf.keras.Input(shape=(dim + 1,), name=transformed_name(key))
        )
    
    for feature in NUMERICAL_FEATURES:
        input_features.append(
            tf.keras.Input(shape=(1,), name=transformed_name(feature))
        )
    
    concatenate = tf.keras.layers.concatenate(input_features)
    deep = concatenate
    
    for i in range(hparams['num_layers']):
        deep = tf.keras.layers.Dense(hparams[f'units_{i}'], activation='relu')(deep)
        dropout_rate = hparams.get(f'dropout_{i}', 0.0)
        if dropout_rate > 0:
            deep = tf.keras.layers.Dropout(dropout_rate)(deep)
    
    outputs = tf.keras.layers.Dense(1, activation="sigmoid")(deep)
    model = tf.keras.models.Model(inputs=input_features, outputs=outputs)
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hparams['learning_rate']),
        loss="binary_crossentropy",
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )
    
    return model

def run_fn(fn_args: FnArgs):
    """Fungsi utama untuk Trainer di TFX."""

    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)
    
    train_dataset = input_fn(fn_args.train_files, tf_transform_output)
    eval_dataset = input_fn(fn_args.eval_files, tf_transform_output)

    best_hps = os.path.join('output', 'best_hyperparameters.json')
    
    with open(best_hps, "r", encoding="utf-8") as f:
        best_hparams = json.load(f)

    model = build_model(best_hparams)
    model.summary()

    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), "logs")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir, update_freq="batch"
    )

    model.fit(
        x=train_dataset,
        epochs=20,
        validation_data=eval_dataset,
        steps_per_epoch=50,
        validation_steps=25,
        callbacks=[tensorboard_callback],
    )

    signatures = {
        "serving_default": get_serve_tf_examples_fn(
            model, tf_transform_output
        ).get_concrete_function(
            tf.TensorSpec(shape=[None], dtype=tf.string, name="examples")
        ),
    }
    model.save(
        fn_args.serving_model_dir, save_format="tf", signatures=signatures
    )

Overwriting depression_trainer.py


Menjalankan proses permodelan

In [21]:
trainer  = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples = transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=1000),
    eval_args=trainer_pb2.EvalArgs(num_steps=500)
)
interactive_context.run(trainer)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Dietary Habits_xf (InputLayer)  [(None, 5)]         0           []                               
                                                                                                  
 Family History of Mental Illne  [(None, 3)]         0           []                               
 ss_xf (InputLayer)                                                                               
                                                                                                  
 Have you ever had suicidal tho  [(None, 3)]         0           []                               
 ughts ?_xf (InputLayer)                                                                          
                                                                                            

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: output\depression-pipeline\Trainer\model\7\Format-Serving\assets


INFO:tensorflow:Assets written to: output\depression-pipeline\Trainer\model\7\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

### Tahapan Analisis dan Evaluasi

Mengambil model terbaru

In [22]:
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
 
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

Mengevaluasi performa model dengan metrik ExampleCount, AUC, MeanPrediction, Precision, Recall, FalsePositives, TruePositives, FalseNegatives, TrueNegatives, dan BinaryAccuracy.

In [23]:
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='Depression')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='Precision'),
            tfma.MetricConfig(class_name='Recall'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]
 
)

Menjalankan proses evaluasi model

In [24]:
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
 
interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

Menampilkan hasil evaluasi model dengan metrik yang telah ditentukan

In [25]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

### Tahap Pusher Model

Melakukan proses pusher untuk model yang telah mendapatkan blessing dari evaluator

In [26]:
pusher = Pusher(
model=trainer.outputs['model'],
model_blessing=evaluator.outputs['blessing'],
push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory=SERVING_MODEL_DIR))
 
)
 
interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))